In [ ]:
# default_exp classify_ions

In [1]:
#export
import numpy as np
import pandas as pd
import scipy.stats
def collect_node_parameters(all_nodes, w_annot = True):
    ion2param2val = {}
    all_headers = set()
    for node in all_nodes:

        param2val = { "frac_mainclust" : node.frac_mainclust, "num_mainclusts" : node.num_mainclusts,
        "cv_fcs" : calc_variance_for_node(node), "num_children" : len(node.children), "fraction_consistent": node.fraction_consistent, "num_leaves" : len(node.leaves), "replicate_cv" : node.cv,
        "intensity" : node.min_intensity}
        if w_annot:
            param2val.update({"positive_example" : node.positive_example, "ion" : node.name})
        if node.type == 'mod_seq_charge':
            if len(node.children) ==2:
                fcfc_diff = abs(node.children[0].fc - node.children[1].fc)
                param2val.update({"ms1_ms2_fcfc_diff" : fcfc_diff})

            for child in node.children:
                name_frac_mainclust = f"child_type_{child.type}_frac_mainclust"
                name_num_mainclusts = f"child_type_{child.type}_num_mainclusts"
                name_variance = f"child_type_{child.type}_cv_fcs"
                name_replicate_cv = f"child_type_{child.type}_replicate_cv"
                param2val.update({name_frac_mainclust: child.frac_mainclust, name_num_mainclusts : child.num_mainclusts, name_variance : calc_variance_for_node(child), name_replicate_cv : child.cv})
                clusterstats_dict = calc_cluster_stats(child)
                param2val.update(clusterstats_dict)
        all_headers.update(param2val.keys())
        ion2param2val[node.name] = param2val

    return get_dataframe(all_nodes,ion2param2val, all_headers)

def get_dataframe(all_nodes,ion2param2val, all_headers):
    all_headers = list(all_headers)
    rows = [[ion2param2val.get(node.name).get(header, np.nan) for header in all_headers] for node in all_nodes]
    df = pd.DataFrame(rows, columns = all_headers)
    return df


def calc_cluster_stats(node):
    num_elems_secondclust = 0
    fcs_clust0 = []
    fcs_clust1 = []
    intensities = []
    for child in node.children:
        if child.cluster ==0:
            fcs_clust0.append(child.fc)
        if child.cluster ==1:
            num_elems_secondclust+=1
            fcs_clust1.append(child.fc)
        intensities.append(child.min_intensity)

    if len(fcs_clust1)>0:
        betweenclust_fcfc = abs(np.mean(fcs_clust0) - np.mean(fcs_clust1))
    else:
        betweenclust_fcfc = 8
    
    stats_dict = {f"child_type_{node.type}_num_elems_secondclust" : num_elems_secondclust, f"child_type_{node.type}_num_clusters_total" : node.num_clusters, f"child_type_{node.type}_betweenclust_fcfc" : betweenclust_fcfc,
    f"child_type_{node.type}_num_elems_secondclust" : np.mean(intensities)}
    return stats_dict

def calc_variance_for_node(node):
    
    fcs_children = [x.fc for x in node.children]
    min_fc = min(fcs_children)
    fcs_children = [x.fc - min_fc for x in node.children]
    #print(f"fcs children are {fcs_children}, variance is {np.var(fcs_children)}")

    return scipy.stats.variation(fcs_children)

In [ ]:
#export
import anytree

def get_positive_negative_samples(condpairtree,type):
    all_nodes = []
    for protein_node in condpairtree.children:
        if protein_node.num_mainclusts <3:
            continue
        if protein_node.fraction_consistent < 0.7:
            continue
        type_nodes = anytree.search.findall(protein_node, filter_=lambda node: node.type == type)
        for type_node in type_nodes:
            fcdiff = abs(type_node.fc - protein_node.fc)

            if fcdiff<0.3:
                type_node.positive_example = True
                all_nodes.append(type_node)
                continue
            if fcdiff>2:
                type_node.positive_example = False
                all_nodes.append(type_node)
    return all_nodes

In [ ]:
#export
from sklearn.utils import resample
import alphaquant.classify_ions as aqclass

def get_tp_fp_nodes(condpairtree, nodetype, substantially_off_thresholds, close_to_target_threshold, precursor2fc = None):
    res_nodes =[]
    condpairtree.type = 'condpair'
    nodes = anytree.search.findall(condpairtree, filter_=lambda node:  (node.type == nodetype))
    if precursor2fc is not None:
        node2fc = get_node2fc(precursor2fc, nodes)

    for node in nodes:
        if precursor2fc is not None:
            fc = node2fc.get(node)
        else:
            fc = node.fc
        if (abs(fc) > substantially_off_thresholds[0]) & (abs(fc) < substantially_off_thresholds[1]):
            node.positive_example = False
            res_nodes.append(node)
        if abs(fc) < close_to_target_threshold:
            node.positive_example = True
            res_nodes.append(node)
    return res_nodes


In [ ]:
#export
import re
def get_node2fc(precursor2fc, nodes):
    node2fc = {}
    pattern = "(SEQ_.*_MOD_)(.*)(_CHARGE_)(.*)(_)"
    if nodes[0].name in precursor2fc.keys():
        rename_precursor= False
    else:
        rename_precursor = True
    for node in nodes:
        if rename_precursor:
            matched = re.search(pattern,node.name)
            precursor = f"{matched.group(2)}.{matched.group(4)}"
        else:
            precursor = node.name
        fc = precursor2fc.get(precursor)
        node2fc[node] = fc

    return node2fc



In [ ]:
#export
import alphaquant.visualizations as aqplot
def generate_ml_input(c1, c2, neg_thresholds =[1.5, 3], pos_threshold =  0.2, features_to_exclude = [], precursor2fc = None, results_folder="results_renamed"):
    tree = aqplot.read_condpair_tree(c1, c2, results_folder)
    pos_negs = get_tp_fp_nodes(tree, 'mod_seq_charge', neg_thresholds, pos_threshold, precursor2fc)
    df_precursor_features = aqclass.collect_node_parameters(pos_negs)
    df_precursor_features = df_precursor_features.drop(columns = features_to_exclude)
    
    df_precursor_features = balance_classes(df_precursor_features)
    #df_precursor_features = df_precursor_features.dropna()
    ionnames = df_precursor_features["ion"]
    df_precursor_features = df_precursor_features.drop(columns = "ion")
    df_precursor_features = df_precursor_features.astype('float')
    df_precursor_features = df_precursor_features.replace(np.nan, -1)
    
    X_outliers = df_precursor_features.drop(columns=["positive_example"]).to_numpy()
    print("shapes:")
    print(X_outliers.shape)
    y_outliers = df_precursor_features["positive_example"]
    print(y_outliers.shape)
    featurenames = list(df_precursor_features.drop(columns=["positive_example"]).columns)
    return X_outliers, y_outliers, featurenames, ionnames



In [ ]:
#export
import anytree
def ml_filter_and_plot_fcs(ml_classifier, threshold_for_negative_classification, c1, c2, results_folder="results_renamed"):
    results_df = aqplot.get_diffresult_dataframe(c1, c2, results_folder=results_folder)
    tree = aqplot.read_condpair_tree(c1, c2, results_folder)
    tree.type = "condpair"
    nodes = anytree.findall(tree, filter_= lambda x : x.type == 'mod_seq_charge')
    ion2isincluded = get_ion2classification(ml_classifier, nodes, threshold_for_negative_classification)
    results_df = results_df[[ion2isincluded.get(x) for x in results_df["protein"]]]

    aqplot.volcano_plot(results_df)



In [ ]:
#export
def get_ion2classification(ml_classifier, nodes, threshold_for_positive_classification):
    df_precursor_features = aqclass.collect_node_parameters(nodes, w_annot=False)
    df_precursor_features = df_precursor_features.astype('float')
    df_precursor_features = df_precursor_features.replace(np.nan, -1)
    X = df_precursor_features.to_numpy()
    ionnames = [x.name for x in nodes]
    probabilities_positive_example = [x[1] for x in ml_classifier.predict_proba(X)]
    ion2included = {ion:(prob>threshold_for_positive_classification ) for ion, prob in zip(ionnames, probabilities_positive_example)}
    return ion2included



In [ ]:
#export

def balance_classes(df_precursor_features):
    df_pos_examples = df_precursor_features[df_precursor_features["positive_example"] ==1]
    df_neg_examples = df_precursor_features[df_precursor_features["positive_example"] ==0]
    min_length = min(len(df_pos_examples.index), len(df_neg_examples.index))
    df_pos_examples = resample(df_pos_examples, replace = False, n_samples = int(min_length), random_state = 123)
    df_neg_examples = resample(df_neg_examples, replace = False, n_samples = min_length, random_state = 123)
    df_downsampled = pd.concat([df_pos_examples, df_neg_examples], ignore_index=True)
    print(sum(df_downsampled["positive_example"]))
    print(len(df_downsampled.index))
    return df_downsampled



In [ ]:
#export
import numpy as np

from sklearn.utils import resample
def load_feature_df(c1, c2, neg_thresholds =[1.5, 3], pos_threshold =  0.2, features_to_exclude = []):
    tree = aqplot.read_condpair_tree(c1, c2, results_folder="results")
    pos_negs = get_tp_fp_nodes(tree, 'mod_seq_charge', neg_thresholds, pos_threshold)
    df_precursor_features = aqclass.collect_node_parameters(pos_negs)
    df_precursor_features = df_precursor_features.drop(columns = features_to_exclude)
    return df_precursor_features

    


In [ ]:
#export
def annotate_feature_df_w_spectronaut(spectronaut_df, parameter_df, groupby_merge_type = 'mean'):
    spectronaut_df["ion"] = pd.Series(["SEQ_" for x in range(len(spectronaut_df.index))]) + spectronaut_df["PEP.StrippedSequence"] + pd.Series(["_MOD_" for x in range(len(spectronaut_df.index))]) + spectronaut_df['EG.ModifiedSequence'] +  pd.Series(["_CHARGE_" for x in range(len(spectronaut_df.index))])+ spectronaut_df['FG.Charge'].astype('str') +pd.Series(["_" for x in range(len(spectronaut_df.index))])
    precursor_headers = ["ion"] + [x for x in spectronaut_df.columns if (("EG." in x) | ("FG." in x))]
    spectronaut_df = spectronaut_df[precursor_headers].drop_duplicates()
    spectronaut_df = spectronaut_df.replace({False: 0, True: 1})
    precursor_headers_final = ["ion"] + list(spectronaut_df.select_dtypes(include=np.number).columns)
    spectronaut_df = spectronaut_df[precursor_headers_final]
    merged_df = parameter_df.merge(spectronaut_df, how = 'left', on = 'ion')
    if groupby_merge_type == 'mean':
        merged_df = merged_df.groupby('ion').mean().reset_index()
    if groupby_merge_type == 'min':
        merged_df = merged_df.groupby('ion').min().reset_index()
    if groupby_merge_type == 'max':
        merged_df = merged_df.groupby('ion').max().reset_index()
    return merged_df



In [ ]:
#export
def reformat_to_ml_input(annotated_df):
    annotated_df = annotated_df.drop(columns = "ion")
    annotated_df = annotated_df.astype('float')
    annotated_df = annotated_df.replace(np.nan, -1)
    #df_precursor_features = df_precursor_features.dropna()
    annotated_df = balance_classes(annotated_df)
    X_outliers = annotated_df.drop(columns=["positive_example"]).to_numpy()
    y_outliers = annotated_df["positive_example"]
    featurenames = list(annotated_df.drop(columns=["positive_example"]).columns)
    return X_outliers, y_outliers, featurenames




In [2]:
#export
from matplotlib import pyplot as plt
import numpy as np

def plot_feature_importances(coef, names, top_n = np.inf, print_out_name = False):
    imp,names = filter_sort_top_n(coef, names, top_n)
    plt.barh(range(len(names)), imp, align='center')
    plt.yticks(range(len(names)), names)
    if print_out_name:
        for idx in range(len(imp)):
            print(f"{imp[idx]}\t{names[idx]}")
    plt.show()


def filter_sort_top_n(imp, names, top_n):
    tuplelist = list(zip(imp, names))
    tuplelist.sort(key = lambda x : abs(x[0]),reverse=True)
    tuplelist = tuplelist[:top_n]
    imp = [x[0] for x in tuplelist]
    names = [x[1] for x in tuplelist]
    return imp, names

In [ ]:
#export
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve
import matplotlib.pyplot as plt
from sklearn.metrics import average_precision_score
import matplotlib.pyplot as plt

def plot_precision_recall(classfier, X_train, y_train, X_test, y_test):
      y_score = classfier.decision_function(X_train)
      average_precision = average_precision_score(y_train, y_score)

      print('Average precision-recall score: {0:0.2f}'.format(
            average_precision))


      disp = plot_precision_recall_curve(classfier, X_test, y_test)
      disp.ax_.set_title('2-class Precision-Recall curve: '
                        'AP={0:0.2f}'.format(average_precision))
      plt.show()



In [ ]:
#export
def get_precursor2fc(cond1, cond2, results_folder):
    result_df = aqviz.get_diffresult_dataframe(cond1, cond2, results_folder)
    result_dict = dict(zip(result_df["protein"], result_df["log2fc"]))
    return result_dict


In [ ]:
#export

def get_nodes_of_type(cond1, cond2, results_folder, node_type = 'mod_seq_charge'):

    tree_sn = aqplot.read_condpair_tree(cond1, cond2, results_folder=results_folder)
    tree_sn.type = "asd"
    return anytree.findall(tree_sn, filter_= lambda x : (x.type == node_type))

In [ ]:
#export
import alphaquant.classify_ions as aqclass
import alphaquant.visualizations as aqplot
import anytree

def generate_protein_shifted_peptide_set(cond1, cond2, results_folder, precursor_cutoff = 2):
    cpair_tree = aqplot.read_condpair_tree(cond1, cond2, results_folder=results_folder)
    cpair_tree.type = "asd"
    protnodes = anytree.findall(cpair_tree, filter_= lambda x : (x.type == "gene"),maxlevel=2)
    normalized_peptides = []
    for prot in protnodes:
        precursors = anytree.findall(prot, filter_= lambda x : (x.type == "mod_seq_charge"))
        if len(precursors)<precursor_cutoff:
            continue
        fc_prot = prot.fc
        for precursor in precursors: #shift every peptide by the protein fold change, this should make peptides from different proteins comparable and enable analyses/ml on this dataset
            precursor.fc = precursor.fc - fc_prot
            normalized_peptides.append(precursor)
        
    return normalized_peptides

In [ ]:
#export

def get_intersect_sn_diann_precursors(cond1, cond2, sn_folder, diann_folder):
    precursor_nodes_diann = get_nodes_of_type(cond1, cond2, diann_folder, node_type = 'mod_seq_charge')
    precursor_nodes_sn = get_nodes_of_type(cond1, cond2, sn_folder, node_type = 'mod_seq_charge')
    precursors_diann = {x.name.replace("_MOD_", "_MOD__").replace("_CHARGE", "__CHARGE") for x in precursor_nodes_diann}
    precursors_sn = {x.name for x in precursor_nodes_sn}
    
    intersect = precursors_diann.intersection(precursors_sn)
    
    return intersect

In [ ]:
#export

def generate_ml_input_regression(df_precursor_features, nodes):
    ion2fc = {x.name: x.fc for x in nodes}
    df_precursor_features = df_precursor_features.dropna()
    df_precursor_features = df_precursor_features[[(x in ion2fc.keys()) for x in df_precursor_features["ion"]]]
    ionnames = list(df_precursor_features["ion"])
    df_precursor_features = df_precursor_features.drop(columns=["ion"])
    df_precursor_features = df_precursor_features.drop(columns=["positive_example"])
    X = df_precursor_features.to_numpy()
    y = [ion2fc.get(ion) for ion in ionnames]
    featurenames = list(df_precursor_features.columns)
    return X, y, featurenames, ionnames

In [ ]:
#export
import pandas as pd

def reduce_spectronaut_output_to_precursor_level(spectronaut_file):
    tableit = pd.read_csv(spectronaut_file, sep = "\t", chunksize=100000)
    tables = []
    for table_df in tableit:

        subset_table = table_df[["R.Label", "FG.Id"]].drop_duplicates()
        table_df = table_df.loc[subset_table.index] #use index to subset
        tables.append(table_df)

    res_df = pd.concat(tables, ignore_index = True)
    res_df = res_df.dropna(axis=1, how='all')
    return res_df

In [ ]:
#export
import matplotlib.pyplot as plt
import numpy as np
import alphaquant.visualizations as aqplot

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import alphaquant.classify_ions as aqclass
from sklearn import datasets, linear_model


def do_linear_regression(X, y, featurenames, ionnames, prediction_cutoff):
    regr = linear_model.LinearRegression()
    X = scale_input(X)
    # Split the data into training/testing sets
    fc2ion = {x:y for x,y in zip(y, ionnames)}
    X_train, X_test, y_train, y_test = train_test_split(X, y)

    # Create linear regression object
    

    # Train the model using the training sets
    regr.fit(X_train, y_train)

    # Make predictions using the testing set
    y_pred = regr.predict(X_test)

    # The coefficients
    print('Coefficients: \n', regr.coef_)
    # The mean squared error
    print('Mean squared error: %.2f'
        % mean_squared_error(y_test, y_pred))
    # The coefficient of determination: 1 is perfect prediction
    print('Coefficient of determination: %.2f'
        % r2_score(y_test, y_pred))

    # Plot outputs
    #plt.plot(X_test[:,-1], y_pred,  color='black')
    plt.scatter(y_test, y_pred, color='blue')
    plt.show()
    aqclass.plot_feature_importances(regr.coef_,featurenames, 10)
    plt.show()
    aqplot.plot_predicted_fc_histogram(y_test, y_pred, prediction_cutoff, show_filtered=False)
    plt.show()
    aqplot.plot_predicted_fc_histogram(y_test, y_pred, prediction_cutoff, show_filtered=True)
    plt.show()
    print_good_predicitions(y_test, y_pred, fc2ion)

import sklearn.preprocessing
def scale_input_minmax(X):
    scaler = sklearn.preprocessing.MinMaxScaler()
    X_transformed = scaler.transform(X)
    return X_transformed

def scale_input(X):
    scaler = sklearn.preprocessing.StandardScaler().fit(X)
    X_scaled = scaler.transform(X)
    return X_scaled

def print_good_predicitions(y_test, y_pred, fc2ionnames,top_n = 10):
    tuples = list(zip(y_test, y_pred))
    tuples.sort(key = lambda x: min(abs(np.array([x[0], x[1]]))), reverse=True)
    print('printing names of some well predicted ions')
    for idx in range(top_n):
        print(f"{tuples[idx]}\t{fc2ionnames.get(tuples[idx][0])}")
        


In [ ]:
#export
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor



def do_random_forest_regression(X, y, featurenames, prediction_cutoff):
      # Split the data into training/testing sets
      X = scale_input(X)
      X_train, X_test, y_train, y_test = train_test_split(X, y)
      
      # Create linear regression object
      regr = RandomForestRegressor(max_depth=2, random_state=0)

      # Train the model using the training sets
      regr.fit(X_train, y_train)

      # Make predictions using the testing set
      y_pred = regr.predict(X_test)


      # The mean squared error
      print('Mean squared error: %.2f'
            % mean_squared_error(y_test, y_pred))
      # The coefficient of determination: 1 is perfect prediction
      print('Coefficient of determination: %.2f'
            % r2_score(y_test, y_pred))
      
      # Plot outputs
      #plt.plot(X_test[:,-1], y_pred,  color='black')
      plt.scatter(y_test, y_pred, color='blue')
      plt.show()
      aqclass.plot_feature_importances(regr.feature_importances_,featurenames, 10)
      plt.show()
      aqplot.plot_predicted_fc_histogram(y_test, y_pred, prediction_cutoff, show_filtered=False)
      plt.show()
      aqplot.plot_predicted_fc_histogram(y_test, y_pred, prediction_cutoff, show_filtered=True)
      plt.show()


In [ ]:
#export
import alphaquant.visualizations as aqplot
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import random


def balance_small_and_strong_fcs(X_orig, y_orig, ionnames,cutoff):
    smaller_pep_idxs = []
    larger_peps_idxs = []
    for idx in range(len(y_orig)):
        fc = y_orig[idx]
        if fc< cutoff:
            smaller_pep_idxs.append(idx)
        else:
            larger_peps_idxs.append(idx)
    if len(larger_peps_idxs)< len(smaller_pep_idxs):
        print("balancing subsets")
        print(f"{len(larger_peps_idxs)} of large vs {len(smaller_pep_idxs)} of small fcs")
        smaller_pep_idxs = random.sample(smaller_pep_idxs, len(larger_peps_idxs))
    else:
        print("skip balancing subsets")
        return X_orig, y_orig, ionnames
    new_idxs = smaller_pep_idxs+larger_peps_idxs
    X_new = X_orig[new_idxs,:]
    y_new = y_orig[new_idxs]
    return X_new, y_new, ionnames[new_idxs]


def random_forest_iterative_cross_predict(X, y, ionnames,number_splits, regr, balancing_cutoff = np.inf):
    y = np.array(y)
    X = scale_input(X)
    X_balanced, y_balanced, ionnames_balanced = balance_small_and_strong_fcs(X, y, ionnames,balancing_cutoff)
    #X_not_balanced, y_not_balanced  = get_balance_excluded_subset(y_balanced, y, X)
    
    chunks_included, chunks_excluded = get_indices_for_cross_predict(y_balanced, number_splits)
    y_test_all = []
    y_pred_all = []
    ionnames_all = []
    all_excluded = {}
    for i in range(len(chunks_included)):
        idxs_in = chunks_included[i]
        idxs_out = chunks_excluded[i]

        if len(set(idxs_out).intersection(all_excluded))>0:
            print("overfitting alarm!")
            all_excluded.update(set(idxs_out))

        X_train = X_balanced[idxs_in,:]
        y_train = y_balanced[idxs_in]
        X_test = X_balanced[idxs_out, :]
        y_test = y_balanced[idxs_out]
        regr.fit(X_train, y_train)
        # Make predictions using the testing set
        y_pred = regr.predict(X_test)
        print(idxs_out[:20])
        y_test_all.extend(y_test)
        y_pred_all.extend(y_pred)
        ionnames_all.extend(ionnames_balanced[idxs_out])
    #predict the ones that were excluded for small fcs (predictor chosen aribtrarly)
    #y_pred_not_balanced = regr.predict(X_not_balanced)
    #y_pred_all.extend(y_pred_not_balanced)
    #y_test_all.extend(y_not_balanced)
    return y_test_all, y_pred_all, ionnames_all
    #aqplot.get_error_and_scatter_ml_regression(y_test_all, y_pred_all, scatter_filt)
        
        

import random
import math

def get_indices_for_cross_predict(y, number_splits):
    length_subset = math.floor(len(y)/number_splits)
    y_idxs = list(range(len(y)))
    random.shuffle(y_idxs)
    chunks_excluded = []
    chunks_included = []
    for i in range(0, len(y), length_subset):
        idxs_excluded = y_idxs[i:i + length_subset]
        idxs_included = [idx for idx in y_idxs if idx not in idxs_excluded]
        chunks_excluded.append(idxs_excluded)
        chunks_included.append(idxs_included)
    return chunks_included, chunks_excluded


def get_balance_excluded_subset(y_balanced, y, X):
    idxs_y_balance_excluded = [x for x in range(len(y)) if (y[x] not in y_balanced)]
    y_not_balanced = np.array(y)[idxs_y_balance_excluded]
    X_not_balanced = X[idxs_y_balance_excluded,:]
    return X_not_balanced, y_not_balanced


def calculate_log_loss_scores_for_prediction(y_test, y_pred):
    loss_scores = abs(np.log2(y_test) - np.log2(y_pred))
    return loss_scores